# E-Mail Spam Classification
## YZV 311E Term Project

Abdullah Bilici, 150200330

Bora Boyacıoğlu, 150200310

Import the necessary libraries.

In [1]:
import pandas as pd
import numpy as np
from utils import *

## Loading the Data

Load the CSV data to a Pandas DataFrame.

In [2]:
mails = pd.read_csv("../Data/emails.csv")

## Text Preprocessing

Using an English NLP model, tokenise the sentences for each mail. Then, apply some rules to make the data workable. These include:

* Tokenisation
1. Lowercasing
2. Stop word removal
3. Special character removal
4. Lemmatisation

Apply tokenisation.

In [3]:
mails['tokenised_text'] = mails['text'].apply(tokenize_sentence)

In [4]:
mails['tokenised_text'].head()

0    [subject, naturally, irresistible, corporate, ...
1    [subject, stock, trading, gunslinger, fanny, m...
2    [subject, unbelievable, new, home, easy, m, wa...
3    [subject, 4, color, printing, special, request...
4    [subject, money, software, cd, software, compa...
Name: tokenised_text, dtype: object

## Feature Modelling

Transforming the text into numerical format and vectorising them

### TF-IDF
TF-IDF considers the frequency of the word in the sentence in relation to the frequency in the corpus, helping to diminish the importance of frequently occurring words in the dataset.

In [5]:
from sklearn.feature_extraction import DictVectorizer

tf = term_frequency(mails["tokenised_text"])
idf = inverse_document_frequency(mails["tokenised_text"])
tfidf = tfidf_calculator(tf,idf)

vectorizer = DictVectorizer(sparse=False)
tfidf_matrix = pd.DataFrame(vectorizer.fit_transform(tfidf), columns=vectorizer.get_feature_names_out())

In [6]:
tfidf_matrix.head()

,,,,,,,,0,00,000,...,zusaetzlich,zuzana,zwabic,zwischen,zwrocic,zwwyw,zygoma,zymg,zzncacst,zzzz
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
# np.savez("../Data/data.npz",data = tfidf_matrix, columns = tfidf_matrix.columns)